In [2]:
import os
import time
import copy
import json
import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, models, transforms
from PIL import Image

# Kural 10 için eklendi:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# --- 1. Kural 4: Random Seed Ayarlama ---
def set_seed(seed=42):
    """Her çalıştırmada aynı sonuçları almak için."""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Birden fazla GPU varsa
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed {seed} olarak ayarlandı.")

# --- 2. Kural 6: Erken Durdurma Sınıfı ---
class EarlyStopper:
    """
    Doğrulama kaybını (validation loss) izler ve erken durdurmayı yönetir.
    En iyi modeli 'checkpoint_path' adresine kaydeder.
    """
    def __init__(self, patience=5, min_delta=0, checkpoint_path='best_model.pth'):
        self.patience = patience # Kural 6: 5 adım
        self.min_delta = min_delta
        self.checkpoint_path = checkpoint_path
        self.counter = 0
        self.best_loss = np.Inf
        self.early_stop = False

    def __call__(self, val_loss, model):
        # Eğer kayıp, en iyi kayıptan daha iyiyse (düşükse)
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            # O anki modelin en iyi ağırlıklarını kaydet
            torch.save(model.state_dict(), self.checkpoint_path)
            # print(f"   -> İyileşme var. Model {self.checkpoint_path} adresine kaydedildi.")
        else:
            # İyileşme yok
            self.counter += 1
            # print(f"   -> İyileşme yok. Sabır: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                print("--- ERKEN DURDURMA TETİKLENDİ ---")
                self.early_stop = True

# --- 3. Veri Yükleme ve Dönüşüm Fonksiyonları ---

def get_transforms(model_name):
    """
    Modele göre uygun dönüşümleri (transform) döndürür.
    Kural 11: Eğitim seti için veri artırma içerir.
    """
    # InceptionNet 299x299, diğerleri 224x224 piksel girdi bekler
    input_size = 299 if model_name == 'inception_v3' else 224
    
    # ImageNet ortalama ve standart sapmaları
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    # Kural 11: Eğitim setine veri artırma uygula
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15), # 15 dereceye kadar döndürme
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1), # Renk ayarı
        transforms.ToTensor(),
        normalize,
    ])

    # Doğrulama ve Test setlerine veri artırma uygulanmaz
    val_test_transform = transforms.Compose([
        transforms.Resize(int(input_size / 0.875)), # 224->256, 299->342
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize,
    ])
    
    return train_transform, val_test_transform

class TransformedSubset(Dataset):
    """
    Veri setini böldükten (random_split) sonra her bir parçaya 
    ayrı ayrı transform uygulayabilmek için bir sarmalayıcı (wrapper) sınıf.
    """
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# --- 4. Kural 1 & 9: Modeli Getirme ve Classifier Değiştirme ---

def get_model(model_name, num_classes, use_pretrained=True):
    """İstenen modeli yükler, dondurur ve classifier'ını değiştirir."""
    model = None
    
    # Tüm katmanları dondur
    def freeze_model(model):
        for param in model.parameters():
            param.requires_grad = False
            
    if model_name == 'mobilenet_v3_small':
        model = models.mobilenet_v3_small(weights='IMAGENET1K_V1' if use_pretrained else None)
        freeze_model(model)
        in_features = model.classifier[0].in_features
        # Kural 9: Yeni classifier (Linear, Dropout, Linear)
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
        
    elif model_name == 'inception_v3':
        model = models.inception_v3(weights='IMAGENET1K_V1' if use_pretrained else None)
        freeze_model(model)
        # Ana classifier
        in_features_fc = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(in_features_fc, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
        # Yardımcı (Aux) classifier (Inception'a özel)
        in_features_aux = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = nn.Sequential(
            nn.Linear(in_features_aux, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='IMAGENET1K_V1' if use_pretrained else None)
        freeze_model(model)
        in_features = model.classifier[1].in_features
        # Kural 9: Yeni classifier
        model.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    else:
        raise ValueError(f"Bilinmeyen model adı: {model_name}")

    return model

# --- 5. Kural 2: Eğitim ve Doğrulama Fonksiyonları ---

def train_one_epoch(model, loader, criterion, optimizer, device, is_inception=False):
    """Bir epoch'luk eğitim işlemi."""
    model.train()  # Modeli eğitim moduna al
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # InceptionV3 eğitimde 2 çıktı verir (ana ve aux)
        if is_inception:
            outputs, aux_outputs = model(inputs)
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux_outputs, labels)
            loss = loss1 + 0.4 * loss2  # Inception standardı
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = running_corrects.double() / len(loader.dataset)
    
    return epoch_loss, epoch_acc.item()

def validate_one_epoch(model, loader, criterion, device):
    """Bir epoch'luk doğrulama (veya test) işlemi."""
    model.eval()  # Modeli değerlendirme moduna al
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():  # Gradyanları kapat
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # InceptionV3 eval modunda tek çıktı verir
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = running_corrects.double() / len(loader.dataset)

    return epoch_loss, epoch_acc.item()

# --- 6. Kural 5: Grafik Çizdirme Fonksiyonu ---

def plot_and_save_curves(history, save_path):
    """Eğitim/Doğrulama kayıp ve doğruluk grafiklerini çizer ve kaydeder."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # Doğruluk (Accuracy) Grafiği
    ax1.plot(history['train_acc'], label='Eğitim Doğruluk')
    ax1.plot(history['val_acc'], label='Doğrulama Doğruluk')
    ax1.set_title(f'Model Doğruluk (En İyi Val: {max(history["val_acc"]):.4f})')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Doğruluk')
    ax1.legend()
    ax1.grid(True)

    # Kayıp (Loss) Grafiği
    ax2.plot(history['train_loss'], label='Eğitim Kayıp')
    ax2.plot(history['val_loss'], label='Doğrulama Kayıp')
    ax2.set_title(f'Model Kaybı (En İyi Val: {min(history["val_loss"]):.4f})')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Kayıp')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Grafikler {save_path} adresine kaydedildi.")
    plt.close(fig)

# --- 7. Kural 10: Karmaşıklık Matrisi Fonksiyonları ---

def get_predictions(model, loader, device):
    """Test/Val loader için tüm tahminleri ve etiketleri döndürür."""
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            # Etiketleri CPU'da tutabiliriz
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

def plot_and_save_confusion_matrix(y_pred, y_true, class_names, save_path):
    """Karmaşıklık matrisini çizer ve kaydeder."""
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
    plt.title(f'Confusion Matrix (Karmaşıklık Matrisi)')
    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Karmaşıklık matrisi {save_path} adresine kaydedildi.")
    plt.close(fig)

# --- 8. Ana Çalıştırma Fonksiyonu ---

def main():
    # --- TEMEL AYARLAR ---
    set_seed(42) # Kural 4
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Kullanılan cihaz: {DEVICE}")

    # Görseldeki klasör yapısına göre
    DATA_PATH = "/kaggle/input/flowers-dataset/flower_photos"
    
    # Kural 1: Modeller
    MODELS_TO_RUN = ['mobilenet_v3_small', 'efficientnet_b0', 'inception_v3']
    
    # Kural 6: Erken durdurma sabrı
    PATIENCE = 5
    NUM_EPOCHS = 50 # Maksimum epoch sayısı
    BATCH_SIZE = 32

    # --- Kural 3 & 4: Veri Setini Bölme ---
    # Transform UYGULAMADAN tüm veriyi yükle
    full_dataset_no_transform = datasets.ImageFolder(DATA_PATH)
    
    NUM_CLASSES = len(full_dataset_no_transform.classes)
    CLASS_NAMES = full_dataset_no_transform.classes
    print(f"Sınıf sayısı: {NUM_CLASSES}, Sınıflar: {CLASS_NAMES}")

    # Boyutları hesapla (%80, %10, %10)
    total_size = len(full_dataset_no_transform)
    train_size = int(0.8 * total_size)
    val_size = int(0.1 * total_size)
    test_size = total_size - train_size - val_size

    print(f"Toplam veri: {total_size}, Eğitim: {train_size}, Doğrulama: {val_size}, Test: {test_size}")

    # Kural 4: random_seed kullanarak böl
    train_subset, val_subset, test_subset = random_split(
        full_dataset_no_transform,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    # --- Karşılaştırma için global değişkenler ---
    # Kural 8: En başarılı modeli takip et (doğrulama kaybına göre)
    best_overall_val_loss = np.Inf
    best_overall_model_name = None
    best_overall_model_path = ""
    
    # --- MODELLERİ SIRAYLA EĞİTME ---
    for model_name in MODELS_TO_RUN:
        print("\n" + "="*50)
        print(f"Model Eğitiliyor: {model_name}")
        print("="*50)

        # Kural 7: Her model için ayrı klasör
        SAVE_DIR = f'./{model_name}_results'
        os.makedirs(SAVE_DIR, exist_ok=True)
        # Her modelin en iyi ağırlığı bu yola kaydedilecek
        CHECKPOINT_PATH = os.path.join(SAVE_DIR, 'best_model_weights.pth')
        
        # Modele özel transformları ve veri yükleyicileri hazırla
        train_transform, val_test_transform = get_transforms(model_name)
        
        train_dataset = TransformedSubset(train_subset, transform=train_transform)
        val_dataset = TransformedSubset(val_subset, transform=val_test_transform)
        
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
        
        # Kural 1 & 9: Modeli al
        model = get_model(model_name, NUM_CLASSES).to(DEVICE)
        
        # Sadece requires_grad=True olanlar (yeni classifier) güncellenecek.
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        
        # Kural 6: Erken durdurucuyu başlat
        early_stopper = EarlyStopper(patience=PATIENCE, checkpoint_path=CHECKPOINT_PATH)
        
        # Eğitim geçmişini tut
        history = {'train_loss': [], 'train_acc': [], 'val_acc': [], 'val_loss': []}

        start_time = time.time()
        
        for epoch in range(NUM_EPOCHS):
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
            
            # Kural 2: Fonksiyonları kullan
            train_loss, train_acc = train_one_epoch(
                model, train_loader, criterion, optimizer, DEVICE,
                is_inception=(model_name == 'inception_v3')
            )
            val_loss, val_acc = validate_one_epoch(
                model, val_loader, criterion, DEVICE
            )
            
            print(f"  Eğitim Kayıp: {train_loss:.4f} | Eğitim Doğruluk: {train_acc:.4f}")
            print(f"  Doğrulama Kayıp: {val_loss:.4f} | Doğrulama Doğruluk: {val_acc:.4f}")

            # Geçmişi kaydet
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            
            # Kural 6: Erken durdurmayı kontrol et
            early_stopper(val_loss, model)
            if early_stopper.early_stop:
                break
        
        end_time = time.time()
        print(f"Eğitim tamamlandı. Toplam süre: {(end_time - start_time) / 60:.2f} dakika")

        # --- EĞİTİM SONRASI KAYIT VE TEST İŞLEMLERİ (Her model için) ---
        
        # Kural 5 & 7: Grafikleri kaydet
        plot_save_path = os.path.join(SAVE_DIR, 'learning_curves.png')
        plot_and_save_curves(history, plot_save_path)
        
        # Bu modelin en iyi ağırlıklarını geri yükle (Erken durdurma sayesinde)
        print(f"En iyi ağırlıklar ({CHECKPOINT_PATH}) yükleniyor...")
        model.load_state_dict(torch.load(CHECKPOINT_PATH))

        # Bu model için Test Veri Yükleyicisini oluştur
        test_dataset = TransformedSubset(test_subset, transform=val_test_transform)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        # Bu modelin Test performansını ölç
        print("Test verisi üzerinde değerlendiriliyor...")
        test_loss, test_acc = validate_one_epoch(model, test_loader, criterion, DEVICE)
        print(f"  Test Kaybı: {test_loss:.4f} | Test Doğruluğu: {test_acc:.4f}")
        
        # Kural 10: Karmaşıklık Matrisini Çiz ve Kaydet
        preds, labels = get_predictions(model, test_loader, DEVICE)
        cm_path = os.path.join(SAVE_DIR, 'confusion_matrix.png')
        plot_and_save_confusion_matrix(preds, labels, CLASS_NAMES, cm_path)

        # Kural 7: Tüm sonuçları JSON olarak kaydet
        history['best_val_loss'] = early_stopper.best_loss
        history['test_loss'] = test_loss
        history['test_acc'] = test_acc
        results_path = os.path.join(SAVE_DIR, 'training_results.json')
        with open(results_path, 'w') as f:
            # Numpy/Torch öğelerini float'a dönüştür
            serializable_history = {k: [float(i) for i in v] if isinstance(v, list) else float(v) 
                                    for k, v in history.items()}
            json.dump(serializable_history, f, indent=4)
        print(f"Eğitim sonuçları {results_path} adresine kaydedildi.")

        # Kural 8: En başarılı modeli global olarak takip et
        if early_stopper.best_loss < best_overall_val_loss:
            best_overall_val_loss = early_stopper.best_loss
            best_overall_model_name = model_name
            best_overall_model_path = CHECKPOINT_PATH
            print(f"--- YENİ EN İYİ MODEL: {model_name} (Val Kaybı: {best_overall_val_loss:.4f}) ---")

    # --- TÜM MODELLER BİTTİKTEN SONRA ---
    print("\n" + "="*50)
    print("Tüm Modellerin Karşılaştırması Tamamlandı")
    print(f"Genel olarak en başarılı model: {best_overall_model_name}")
    print(f"En iyi doğrulama kaybı: {best_overall_val_loss:.4f}")
    
    # Kural 8: En başarılı modelin ağırlıklarını ana dizine kopyala
    final_best_path = 'BEST_OVERALL_MODEL.pth'
    copyfile(best_overall_model_path, final_best_path)
    print(f"En başarılı modelin ağırlıkları '{final_best_path}' olarak kaydedildi.")
    print("\n--- İşlem Tamamlandı ---")



main()

Random seed 42 olarak ayarlandı.
Kullanılan cihaz: cuda


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


Sınıf sayısı: 5, Sınıflar: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
Toplam veri: 3670, Eğitim: 2936, Doğrulama: 367, Test: 367

Model Eğitiliyor: mobilenet_v3_small


100%|██████████| 9.83M/9.83M [00:00<00:00, 72.9MB/s]


Epoch 1/50
  Eğitim Kayıp: 0.8695 | Eğitim Doğruluk: 0.6839
  Doğrulama Kayıp: 0.4192 | Doğrulama Doğruluk: 0.8556
Epoch 2/50
  Eğitim Kayıp: 0.5938 | Eğitim Doğruluk: 0.7847
  Doğrulama Kayıp: 0.3647 | Doğrulama Doğruluk: 0.8665
Epoch 3/50
  Eğitim Kayıp: 0.5262 | Eğitim Doğruluk: 0.8195
  Doğrulama Kayıp: 0.3540 | Doğrulama Doğruluk: 0.8556
Epoch 4/50
  Eğitim Kayıp: 0.4906 | Eğitim Doğruluk: 0.8178
  Doğrulama Kayıp: 0.3252 | Doğrulama Doğruluk: 0.8692
Epoch 5/50
  Eğitim Kayıp: 0.4769 | Eğitim Doğruluk: 0.8287
  Doğrulama Kayıp: 0.3318 | Doğrulama Doğruluk: 0.8747
Epoch 6/50
  Eğitim Kayıp: 0.4563 | Eğitim Doğruluk: 0.8202
  Doğrulama Kayıp: 0.3143 | Doğrulama Doğruluk: 0.8828
Epoch 7/50
  Eğitim Kayıp: 0.4294 | Eğitim Doğruluk: 0.8399
  Doğrulama Kayıp: 0.2892 | Doğrulama Doğruluk: 0.8965
Epoch 8/50
  Eğitim Kayıp: 0.4508 | Eğitim Doğruluk: 0.8348
  Doğrulama Kayıp: 0.2912 | Doğrulama Doğruluk: 0.8965
Epoch 9/50
  Eğitim Kayıp: 0.4294 | Eğitim Doğruluk: 0.8423
  Doğrulama Kayıp: 0

...

Karmaşıklık matrisi ./mobilenet_v3_small_results/confusion_matrix.png adresine kaydedildi.
Eğitim sonuçları ./mobilenet_v3_small_results/training_results.json adresine kaydedildi.
--- YENİ EN İYİ MODEL: mobilenet_v3_small (Val Kaybı: 0.2270) ---

Model Eğitiliyor: efficientnet_b0


100%|██████████| 20.5M/20.5M [00:00<00:00, 150MB/s]


Epoch 1/50
  Eğitim Kayıp: 0.7192 | Eğitim Doğruluk: 0.7490
  Doğrulama Kayıp: 0.3648 | Doğrulama Doğruluk: 0.8692
Epoch 2/50
  Eğitim Kayıp: 0.5037 | Eğitim Doğruluk: 0.8181
  Doğrulama Kayıp: 0.3402 | Doğrulama Doğruluk: 0.8828
Epoch 3/50
  Eğitim Kayıp: 0.4692 | Eğitim Doğruluk: 0.8321
  Doğrulama Kayıp: 0.2882 | Doğrulama Doğruluk: 0.8910
Epoch 4/50
  Eğitim Kayıp: 0.4043 | Eğitim Doğruluk: 0.8556
  Doğrulama Kayıp: 0.2883 | Doğrulama Doğruluk: 0.8910
Epoch 5/50
  Eğitim Kayıp: 0.3983 | Eğitim Doğruluk: 0.8590
  Doğrulama Kayıp: 0.2800 | Doğrulama Doğruluk: 0.8883
Epoch 6/50
  Eğitim Kayıp: 0.3785 | Eğitim Doğruluk: 0.8661
  Doğrulama Kayıp: 0.2748 | Doğrulama Doğruluk: 0.9101
Epoch 7/50
  Eğitim Kayıp: 0.3735 | Eğitim Doğruluk: 0.8627
  Doğrulama Kayıp: 0.2625 | Doğrulama Doğruluk: 0.9046
Epoch 8/50
  Eğitim Kayıp: 0.3493 | Eğitim Doğruluk: 0.8757
  Doğrulama Kayıp: 0.2558 | Doğrulama Doğruluk: 0.9101
Epoch 9/50
  Eğitim Kayıp: 0.3511 | Eğitim Doğruluk: 0.8726
  Doğrulama Kayıp: 0

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 206MB/s] 


Epoch 1/50
  Eğitim Kayıp: 1.3749 | Eğitim Doğruluk: 0.5909
  Doğrulama Kayıp: 0.5925 | Doğrulama Doğruluk: 0.7875
Epoch 2/50
  Eğitim Kayıp: 0.9548 | Eğitim Doğruluk: 0.7432
  Doğrulama Kayıp: 0.4713 | Doğrulama Doğruluk: 0.8202
Epoch 3/50
  Eğitim Kayıp: 0.9336 | Eğitim Doğruluk: 0.7360
  Doğrulama Kayıp: 0.4470 | Doğrulama Doğruluk: 0.8392
Epoch 4/50
  Eğitim Kayıp: 0.8491 | Eğitim Doğruluk: 0.7575
  Doğrulama Kayıp: 0.4315 | Doğrulama Doğruluk: 0.8474
Epoch 5/50
  Eğitim Kayıp: 0.8527 | Eğitim Doğruluk: 0.7606
  Doğrulama Kayıp: 0.4602 | Doğrulama Doğruluk: 0.8202
Epoch 6/50
  Eğitim Kayıp: 0.7968 | Eğitim Doğruluk: 0.7694
  Doğrulama Kayıp: 0.4066 | Doğrulama Doğruluk: 0.8447
Epoch 7/50
  Eğitim Kayıp: 0.8431 | Eğitim Doğruluk: 0.7602
  Doğrulama Kayıp: 0.4060 | Doğrulama Doğruluk: 0.8665
Epoch 8/50
  Eğitim Kayıp: 0.7926 | Eğitim Doğruluk: 0.7691
  Doğrulama Kayıp: 0.3927 | Doğrulama Doğruluk: 0.8420
Epoch 9/50
  Eğitim Kayıp: 0.7804 | Eğitim Doğruluk: 0.7772
  Doğrulama Kayıp: 0

In [3]:
!zip -r results.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/BEST_OVERALL_MODEL.pth (deflated 9%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/mobilenet_v3_small_results/ (stored 0%)
  adding: kaggle/working/mobilenet_v3_small_results/learning_curves.png (deflated 8%)
  adding: kaggle/working/mobilenet_v3_small_results/best_model_weights.pth (deflated 9%)
  adding: kaggle/working/mobilenet_v3_small_results/confusion_matrix.png (deflated 22%)
  adding: kaggle/working/mobilenet_v3_small_results/training_results.json (deflated 70%)
  adding: kaggle/working/inception_v3_results/ (stored 0%)
  adding: kaggle/working/inception_v3_results/learning_curves.png (deflated 8%)
  adding: kaggle/working/inception_v3_results/best_model_weights.pth (deflated 7%)
  adding: kaggle/working/inception_v3_results/confusion_matrix.png (deflated 22%)
  adding: kaggle/working/inception_v3_results/training_results.json (deflated 71%)
  adding: kaggle/working/efficientnet_b

In [4]:
from IPython.display import FileLink
FileLink(r'results.zip')

/kaggle/working/results.zip